In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
# %pip install transformers==4.30
%pip install -U peft
%pip install -U accelerate einops
%pip install -U trl 
%pip install rouge_score rouge
%pip install datasets==2.11
%pip install -U flash-attn --no-build-isolation


# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
# !pip install -q -U trl transformers accelerate 
# # !pip install git+https://github.com/huggingface/peft.git
# !pip install -q einops wandb
# %pip install datasets==2.11

# !pip install scipy

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
import datasets
from trl import SFTTrainer
import pandas as pd
import shutil

device = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
# !python sparseml/src/sparseml/transformers/sparsification/obcq/export.py --task text-generation --model_path obcq_deployment --sequence_length 4096
# !cp deployment/model.onnx deployment/model-orig.onnx

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("Wandb")
!huggingface-cli login --token $secret_hf

# Monitering the LLM
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning MPhi SCQA', 
    job_type="training", 
    anonymous="allow"
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rebitib636. Use `wandb login --relogin` to force relogin


In [2]:
sc_data =load_dataset("csv", data_files = "/kaggle/input/scqa-data/Final_SCPairs.csv", split="train[:]", encoding='latin')

train_qa = load_dataset("csv", data_files = "/kaggle/input/scqa-data/FInal_QAPairs.csv", split="train[:]")
test_qa = load_dataset("csv", data_files = "/kaggle/input/scqa-data/QA_test.csv", split="train[:]")

from datasets import Dataset
def preprocess_data(data_dict, is_qa=True):
    final_text = []
    if is_qa:
        for q, a in zip(data_dict['question'],data_dict['answer']):
#             t = f"<s>[INST] Answer the following question truthfully: {q} [/INST] {a} </s>"
#             t = f"<|prompter|>Answer the following question truthfully: {q}</s><|assistant|> {a}"
            t = f"question: {q} answer: {a}"
            final_text.append({'text': t})
    else:
        for q, a in zip(data_dict['Question'],data_dict['Answer']):
#             t = f"<s>[INST] Complete the following text: {q} [/INST] {a} </s>"
#             t = f"<|prompter|>Complete the following text: {q}</s><|assistant|> {a}" 
            t = f"question: {q} answer: {a}"
            final_text.append({'text': t})
    return final_text

train_qa = Dataset.from_list(preprocess_data(train_qa)).with_format("torch")
test_qa = Dataset.from_list(preprocess_data(test_qa)).with_format("torch")

sc_data = Dataset.from_list(preprocess_data(sc_data, is_qa=False)).train_test_split(test_size=0.025).with_format("torch")

print("Test Sentence completion:\n", sc_data['test'][0], end= "\n" + 20* '--' + "\n")
print("Train Sentence completion:\n", sc_data['train'][0], end= "\n" + 20* '--' + "\n")
print("Train Question answer:\n", train_qa[0], end= "\n" + 20* '--' + "\n")
print("Test Question answer:\n", test_qa[0], end= "\n" + 20* '--' + "\n")

print("Test SC: ", len(sc_data['test']))
print("Train SC: ", len(sc_data['train']))
print("Train QA: ", len(train_qa))
print("Test QA: ", len(test_qa))

for i in range(10):
    torch.cuda.empty_cache()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e6b2c5048bec06d7/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9fc4df6906c3d98e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c93aa09ed6e1f934/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
Test Sentence completion:
 {'text': 'question: Were there any scholars whose classes stood out to you answer: I really did enjoy every course I took  and learned from each of my teachers'}
----------------------------------------
Train Sentence completion:
 {'text': 'question: Students may apply to the S. Daniel Abraham Honors Program at Stern College for Women, the Jay and Jeanie Schottenstein Honors Program at Yeshiva College, or the Business Honors and Entrepreneurial Leadership Program at Sy Syms School of Business either directly from high school or while engaged in post-high-school study in Israel. answer: Honors students participate in advanced-level courses, engage in unique extra-curricular activities, complete an intensive senior thesis/project, and are eligible for 

In [3]:
combined = sc_data['train']['text'] + train_qa['text']
list_of_dicts = [{"text": item} for item in combined]

dataset = datasets.Dataset.from_list(list_of_dicts).with_format("torch")
combined_train = dataset.shuffle(seed=42).with_format("torch")
len(combined_train['text']) == len(train_qa['text'] ) + len(sc_data['train']['text'])

combined = sc_data['test']['text'] + test_qa['text']
list_of_dicts = [{"text": item} for item in combined]

dataset = datasets.Dataset.from_list(list_of_dicts).with_format("torch")
combined_test = dataset.shuffle(seed=42).with_format("torch")
len(combined_test['text']) == len(test_qa['text'] ) + len(sc_data['test']['text'])

True

In [4]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
        target_modules = [ "fc1", 'fc2', 'linear', "Wqkv", "out_proj"]

)

In [12]:
%%time
# model_id = "codegood/Mistral_instruct_latest"
model_id = "microsoft/phi-1_5"
model_id = "codegood/MPhi_Latest"
# model_id = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"

model = AutoModelForCausalLM.from_pretrained(
        model_id,
    load_in_4bit = True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
#         use_flash_attention_2=True,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, device_map="auto")
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)
for i in range(10):
    torch.cuda.empty_cache()
    
#Adding the adapters in the layers
model = get_peft_model(model, peft_config)

TypeError: LoraConfig.__init__() got an unexpected keyword argument 'loftq_config'

In [8]:
# Fine tuned model
from peft import PeftConfig
PEFT_MODEL_ID = "codegood/MPhi_Latest"
# PEFT_MODEL_ID = "codegood/Mistral_instruct_latest"


config = PeftConfig.from_pretrained(PEFT_MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit = True,
    return_dict=True,
    quantization_config=bnb_config,
    device_map= "auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(model, PEFT_MODEL_ID)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,  device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

for i in range(10):
    torch.cuda.empty_cache()

In [9]:
per_device_train_batch_size = 1 # reduce batch size by 2x if out-of-memory error
per_device_eval_batch_size = 1
gradient_accumulation_steps = 1  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 1000 # number of updates steps before two checkpoint saves
logging_steps = 500 # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = -1      # training will happen for 320 steps
warmup_ratio = 0.05 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    save_total_limit = 3,
    num_train_epochs=1,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    evaluation_strategy='steps',
    eval_steps=save_steps,
    metric_for_best_model="eval_loss",
    logging_steps=logging_steps,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
#     hub_model_id="HF_AWS_Mistral_SC",
    report_to="tensorboard",
    load_best_model_at_end=True,
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    eval_dataset=sc_data['test'],
#     eval_dataset = combined_test,
    train_dataset=sc_data['train'],
#     train_dataset = combined_train,
    neftune_noise_alpha=5,
    peft_config=peft_config,
    max_seq_length= 256,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:205: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/5462 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

### Sentence completion training

In [10]:
for i in range(10):
    torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
model.push_to_hub("codegood/results")

In [ ]:
check_dir = "/kaggle/working/results"
for i in sorted(os.listdir(check_dir))[:-2]:
    shutil.rmtree(os.path.join(check_dir, i))

In [ ]:
shutil.rmtree("/kaggle/working/")

In [ ]:
from IPython.display import FileLink

tuned_model = "codegood/MPhi_New"

trainer.model.config.save_pretrained(tuned_model )
trainer.save_model(tuned_model)
torch.save(model.merge_and_unload().state_dict(), f"/kaggle/working/{tuned_model}/pytorch_model.bin")
model.config.use_cache = True
model.eval()
# wandb.finish()

# trainer.push_to_hub(tuned_model)
model.push_to_hub(tuned_model)
tokenizer.push_to_hub(tuned_model)
print("Model saved to Huggingface")


# torch.cuda.empty_cache()
# torch.cuda.synchronize()
# trainer.evaluate(metric_key_prefix="rouge")

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/kaggle/working/codegood/MPhi_New",
    repo_id="codegood/MPhi_New",
    repo_type="model",
)

In [ ]:
prompt = "Yeshiva University is"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

In [ ]:
prompt = "Complete the following statement: AI program at Yeshiva University is"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "Complete the following statement: Program director for MS AI program at Yeshiva is"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
%%time
import random
from rouge_score import rouge_scorer
index = random.randint(0, len(sc_data))
# sys_prompt = """ """

tests, testc = sc_data['test']['text'][index].split("[/INST]")
print("Sentence:\n", tests)
print("Ground truth completion:\n", testc)

gen_ans = pipe(f"{tests} [/INST]")[0]['generated_text'].split("[/INST]")[1]
print("Generated answer:", gen_ans)

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)
scores = scorer.score(testc, gen_ans)
scores

In [ ]:
%%time
import random
from rouge_score import rouge_scorer
index = random.randint(0, len(sc_data))
# sys_prompt = """ """

tests, testc = sc_data['train']['text'][index].split("[/INST]")
print("Sentence:\n", tests)
print("Ground truth completion:\n", testc)

gen_ans = pipe(f"{tests} [/INST]")[0]['generated_text'].split("[/INST]")[1]
print("Generated answer:", gen_ans)

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)
scores = scorer.score(testc, gen_ans)
scores

In [ ]:
from tqdm import tqdm
def create_df(data, is_qa_data = True):
    answers, questions, preds = [], [], []
    rouge1_f, rouge1_prec, rouge1_rec = [], [], []
    rouge2_f, rouge2_prec, rouge2_rec = [], [], []
    rougeL_f, rougeL_prec, rougeL_rec = [], [], []

    for qa in tqdm(data):
        torch.cuda.empty_cache()
        dat = qa.split("[/INST]")
        questions.append(dat[0].split("[INST]")[1].split(":")[1].strip())
        answers.append(dat[1][:-4].strip())
        preds.append(pipe(f"<s>[INST]{dat[0]}[/INST]", pad_token_id=tokenizer.eos_token_id)[0]['generated_text'].split("[/INST]")[1])
        scores = scorer.score(questions[-1], preds[-1])
        rouge1_prec.append(scores['rouge1'].precision)
        rouge1_f.append(scores['rouge1'].fmeasure)
        rouge1_rec.append(scores['rouge1'].recall)
        rouge2_prec.append(scores['rouge2'].precision)
        rouge2_f.append(scores['rouge2'].fmeasure)
        rouge2_rec.append(scores['rouge2'].recall)
        rougeL_prec.append(scores['rougeL'].precision)
        rougeL_f.append(scores['rougeL'].fmeasure)
        rougeL_rec.append(scores['rougeL'].recall)

    df = pd.DataFrame({"Question":questions, "Answer": answers, "Prediction": preds, "RLFscore": rougeL_f, "R2Fscore": rouge2_f, "R1Fscore": rouge1_f,\
                    "RLPrec": rougeL_prec, "R2Prec": rouge2_prec, "R1Prec": rouge1_prec, "RLrecall": rougeL_rec, "R2Recall": rouge2_rec, "R1Recall": rouge1_rec})
    return df

# sc_train_res = create_df(sc_data['train']['text'][:60])
sc_test_res = create_df(sc_data['test']['text'][:100])

In [ ]:
# sc_train_res.to_csv("SC_train_res.csv", index=False)
sc_test_res.to_csv("SC_test_res.csv", index=False)

print("Sentence completion results")
print("Test mean F measures")
print("RougeL mean F measure: ", sc_test_res.RLFscore.mean())
print("Rouge1 mean F measure: ", sc_test_res.R1Fscore.mean())
print("Rouge2 mean F measure: ", sc_test_res.R2Fscore.mean())
# print("*"*50)
# print("Train mean F measures")
# print("RougeL mean F measure: ", sc_train_res.RLFscore.mean())
# print("Rouge1 mean F measure: ", sc_train_res.R1Fscore.mean())
# print("Rouge2 mean F measure: ", sc_train_res.R2Fscore.mean())

### Question answer training

In [ ]:
import wandb
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning Mistral Instruct 7B QA', 
    job_type="training", 
    anonymous="allow"
)

In [15]:
per_device_train_batch_size = 5# reduce batch size by 2x if out-of-memory error
per_device_eval_batch_size = 5
gradient_accumulation_steps = 1  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 500 # number of updates steps before two checkpoint saves
logging_steps = 100 # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = -1     # training will happen for 320 steps
warmup_ratio = 0.05 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    evaluation_strategy='steps',
    save_total_limit = 3,
    num_train_epochs = 1,
    eval_steps=save_steps,
    metric_for_best_model="eval_loss",
    logging_steps=logging_steps,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
)

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    eval_dataset=test_qa,
    train_dataset=train_qa,
    neftune_noise_alpha=5,
    peft_config=peft_config,
    max_seq_length= 256,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)


/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:205: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/5166 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [16]:
for i in range(5):
    torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [81]:
model.config.use_cache = True
trainer.evaluate(metric_key_prefix="rouge")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'rouge_loss': 1.0005252361297607,
 'rouge_runtime': 79.9088,
 'rouge_samples_per_second': 12.514,
 'rouge_steps_per_second': 1.251}

In [ ]:
check_dir = "/kaggle/working/results"
for i in sorted(os.listdir(check_dir))[:-2]:
    shutil.rmtree(os.path.join(check_dir, i))

In [98]:
os.chdir("/kaggle/working/")

In [28]:
base_model, new_model = "microsoft/phi-1_5", "codegood/MPhi_New"
model.config.use_cache = True
model.eval()

trainer.model.save_pretrained(new_model)
torch.save(model.merge_and_unload().state_dict(), new_model+"/pytorch_model.bin")

del model, trainer
for i in range(20):
    torch.cuda.empty_cache()

In [29]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True, trust_remote_code=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model_reload, new_model)
model = model.merge_and_unload()
print("Model merged")
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# os.chdir("codegood")
model.push_to_hub(new_model, check_pr=True)
tokenizer.push_to_hub(new_model, check_pr=True)
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/kaggle/working/codegood/MPhi_New",
    repo_id="codegood/MPhi_Latest",
    repo_type="model",
)

Model merged


model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/53.8M [00:00<?, ?B/s]

'https://huggingface.co/codegood/MPhi_Latest/tree/main/'

In [104]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

toke = AutoTokenizer.from_pretrained("codegood/MPhi_New")
mdl = AutoModelForCausalLM.from_pretrained("codegood/MPhi_New", trust_remote_code=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 14.76 GiB total capacity; 13.50 GiB already allocated; 5.75 MiB free; 13.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/kaggle/working/codegood/MPhi_New",
    repo_id="codegood/MPhi_New",
    repo_type="model",
)

In [37]:
import random
from rouge_score import rouge_scorer

prompt = "Does Yeshiva university has MS AI program?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200,pad_token_id=tokenizer.eos_token_id)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Does Yeshiva university has MS AI program? answer: Yeshiva University has a MS in Artificial Intelligence program. The program is designed to provide students with a comprehensive understanding of AI's applications, ranging from applications in business and finance to applications in science and engineering. The program is fully accredited by the New York State Education Department. The MS in AI is a STEM-approved degree, and students can pursue their careers in business, finance, science, engineering, and other fields. The program is fully online, allowing students to engage with the material at their own pace. The faculty includes experts in the field, including Dr. David Sable, Dr. David Freedman, Dr. David Sable, Dr. David Freedman, Dr. David Sable, Dr. David Sable, Dr. David Sable, Dr. David Sable, Dr. David Sable, Dr. David Sable, Dr. David Sable, Dr. David Sable, Dr. David S


In [38]:
prompt = "What are the requirements for Yeshiva University's MS AI program?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What are the requirements for Yeshiva University's MS AI program? [/INST] Yeshiva University's MS AI program requires a minimum of 84 credits, including courses in foundational AI and machine learning, as well as specialized courses in neural networks, reinforcement learning, and large language models. answer: Applicants must have a minimum GPA of 3.2 and a minimum SAT score of 1340 or an ACT score of 29 or 32. They should also demonstrate proficiency in Python, Java, C/C++, and SQL/NoSQL. The program's commitment to excellence is reflected in its rigorous academic standards. Applicants interested in the program should submit their admissions application by March 2023. The program's focus on practical skills prepares students for real-world challenges in AI and machine learning. The curriculum covers topics such as data science, programming, and AI ethics, ensuring a well-rounded education. The program's emphasis on hands-on experience, including internships and


In [39]:
prompt = "List out the requirements for Yeshiva University's MS AI program?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] List out the requirements for Yeshiva University's MS AI program? [/INST] Yeshiva University's MS AI program requires a minimum of 128 credits, with courses including foundational courses, AI Track, and specialized courses. answer: The program also offers a minimum of 12 credits of AI-related electives. Students can progress through the program at a pace that suits their needs. The program's focus on AI and machine learning prepares students for careers in technology, business, finance, and academia. The program's emphasis on AI and machine learning reflects the program's commitment to staying at the forefront of AI and machine learning technologies. The program's focus on AI and machine learning reflects the program's commitment to preparing students for careers in technology, business, finance, and academia. The program's focus on AI and machine learning reflects the program's commitment to preparing students for careers in technology, business, finance, and academia. The p

In [42]:
%%time
prompt = "How to get accepted to Yeshiva University's MS AI program?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200,pad_token_id=tokenizer.eos_token_id)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

How to get accepted to Yeshiva University's MS AI program? High school students with a minimum GPA of 85 and who are accepted to high school may apply. answer: Students who wish to pursue a professional career in AI can opt for the M.S. in Artificial Intelligence and Neural Systems Engineering at the Katz School. The program is designed to provide students with a strong foundation in AI technology and skills, preparing them for leadership roles in the field. The curriculum covers advanced topics in AI, including computer vision, reinforcement learning, and responsible AI. Students engage in hands-on projects, including building AI-powered drones, drones for environmental monitoring, and drones for package delivery. The program also offers a unique opportunity for students to work on AI-related internships, gaining industry experience. The curriculum is structured to be completed in one or two years, providing a fast and efficient pathway to a professional in AI. Students can choose bet

In [ ]:
%%time
import random
index = random.randint(0, len(test_qa))
testQ, testA = test_qa['text'][index].split("[/INST]")
print("Question:\n", testQ)
print("Ground truth completion:\n", testA)

gen_ans = pipe(f"{testQ}")[0]['generated_text'].split("[/INST]")[1]
print("Generated answer:", gen_ans)

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)
scores = scorer.score(testA, gen_ans)
scores

In [ ]:
%%time
index = random.randint(0, len(test_qa))
# sys_prompt = """ """
testQ, testA = train_qa['text'][index].split("[/INST]")
print("Question:\n", testQ)
print("Ground truth completion:\n", testA)

gen_ans = pipe(f"{testQ}")[0]['generated_text'].split("[/INST]")[1]
print("Generated answer:", gen_ans)

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)
scores = scorer.score(testA, gen_ans)
scores

In [ ]:
%%time
testQ = "Can you explain Independent study course in MS AI program?"
print("Question: ", testQ)

gen_ans = pipe(f"<s>[INST]{testQ}[/INST]")[0]['generated_text'].split("[/INST]")[1]
print("Generated answer:", gen_ans)

In [ ]:
# qa_train_res = create_df(train_qa['text'][:60])
# qa_test_res = create_df(test_qa['text'])

In [ ]:
# test = test_qa.to_pandas()
# test['question'] = test.text.apply(lambda x:x.split("[/INST]")[0].split("[INST]")[1].split(":")[1])
# test['answer'] = test['text'].apply(lambda x:x.split("[/INST]")[1])

In [43]:
test = test_qa.to_pandas()
test['question'] = test.text.apply(lambda x:x.split(":")[1][:-6])
test['answer'] = test['text'].apply(lambda x:x.split(":")[2])

In [46]:
torch.cuda.empty_cache()

In [47]:
%%time
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200,pad_token_id=tokenizer.eos_token_id, batch_size=15, trust_remote_code=True)
questions = test.question.apply(lambda x:f"{x}").to_list()
generated_ans = pipe(questions)

CPU times: user 14min 38s, sys: 5min 6s, total: 19min 44s
Wall time: 19min 43s


In [68]:
generated_answers = [i[0]['generated_text'].split(":")[-1] for i in generated_ans]

In [78]:
test['generated'] = generated_answers
test = test[test['generated'] != ""]
test.to_csv("Test_preds_MPhi.csv", index=False)

In [77]:
from rouge import Rouge
from tabulate import tabulate
rouge = Rouge()

scores = rouge.get_scores(test.answer.to_list(), test.generated.to_list(), avg=True)

score_table = [{'Metric': metric, 'Precision': score['p'], 'Recall': score['r'], 'F1-Score': score['f']} for metric, score in scores.items()]

print(tabulate(score_table, headers='keys', tablefmt='grid'))

+----------+-------------+-----------+------------+
| Metric   |   Precision |    Recall |   F1-Score |
+==========+=============+===========+============+
| rouge-1  |    0.393863 | 0.147787  |  0.195931  |
+----------+-------------+-----------+------------+
| rouge-2  |    0.162256 | 0.0440985 |  0.0620082 |
+----------+-------------+-----------+------------+
| rouge-l  |    0.377279 | 0.140092  |  0.186507  |
+----------+-------------+-----------+------------+


In [ ]:
qa_test_res.to_csv("QA_test_res.csv", index=False)
qa_train_res.to_csv("QA_train_res.csv", index=False)

In [ ]:
# test['generated'] = test.generated.apply(lambda x: x[0]['generated_text'].split("[/INST]")[1])
from rouge import Rouge
from tabulate import tabulate

act_ans = []
pred_ans = []

for index, row in test.iterrows():
    act_ans.append(row['answer'])
    pred_ans.append(row['generated'])

hyps, refs = map(list, (pred_ans,act_ans))
rouge = Rouge()

scores = rouge.get_scores(hyps, refs, avg=True)

score_table = [{'Metric': metric, 'Precision': score['p'], 'Recall': score['r'], 'F1-Score': score['f']} for metric, score in scores.items()]

print(tabulate(score_table, headers='keys', tablefmt='grid'))

In [ ]:
test.generated = test.generated.apply(lambda x: x[0]['generated_text'].split("[/INST]")[1])

In [ ]:
import os
import torch
from sparseml.pytorch.models import mnist_net
from sparseml.pytorch.utils import ModuleExporter

exporter = ModuleExporter(model, output_dir=os.path.join(".", "onnx-export"))
exporter.export_onnx(sample_batch=torch.randn(4, 256, dtype=torch.int))